In [2]:
import pandas as pd
import numpy as np
from pgmpy.models import DiscreteBayesianNetwork

# Ad Data Preprocessing

In [29]:
# Define column renaming mapping (adjust if your CSV headers differ slightly)
rename_map = {
        'Device platform': 'Device_platform',
        'Platform': 'Platform',
        'Custom Audience Defined': 'Use_Custom_Audience',
        'Exclusion Defined': 'Use_Exclusions',
        'Amount spent': 'Amount_Spent',
        'CPM (cost per 1,000 impressions)': 'CPM',
        'Clicks (all)': 'Clicks',
        'Leads': 'Number_of_Leads',
        'Cost per Lead': 'CPL',
        'Objective': 'Campaign_Objective',
        'Headline_Local': 'Headline_Local',
        'Headline_Event': 'Headline_Event',
        'Headline_Exclusivity': 'Headline_Exclusivity',
        'Headline_Rental': 'Headline_Rental',
        'Headline_Returns': 'Headline_Returns',
        'Headline_Discounts': 'Headline_Discounts'
}

# Define all columns to keep based on rename_map keys + impressions/clicks
cols_to_keep_original = list(rename_map.keys()) + ['Impressions', 'Reach', 'Frequency', 'Currency'] # Clicks is already in rename_map keys

df = pd.read_csv('./Data/data.csv')
# Select and rename
data = df[cols_to_keep_original].rename(columns=rename_map)

In [30]:
#Calculate CTR

data['Clicks'].fillna(0)
data['Impressions'].fillna(0)
data['Clicks'] = data['Clicks'].astype(int)
data['Impressions'] = data['Impressions'].astype(int)
data['CTR'] = data['Clicks'] / data['Impressions']*100
data['CTR'] = data['CTR'].fillna(0)


#Convert Y/N to 1/0
cols_to_convert = ['Use_Custom_Audience', 'Use_Exclusions']
data[cols_to_convert] = data[cols_to_convert].replace({'Y': 1, 'N': 0})

# Set CPL to 100000 if lead count is 0
condition = (data['Number_of_Leads'] == 0) | pd.isna(data['Number_of_Leads'])
data['CPL'] = np.where(condition, 100000, data['Amount_Spent'] / data['Number_of_Leads'])

# Standardize amount to SGD
data['Amount_Spent'] = np.where(data['Currency'] == 'MYR', data['Amount_Spent'] * 0.3, data['Amount_Spent'])
#data.head

C:\Users\weikang.goh\AppData\Local\Temp\ipykernel_21184\4134718815.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[cols_to_convert] = data[cols_to_convert].replace({'Y': 1, 'N': 0})


In [31]:
# 1. CPM (Percentiles)
# Let's use 5 bins (quintiles: 0-20%, 20-40%, 40-60%, 60-80%, 80-100%).

num_cpm_bins = 5
cpm_labels = [f'Quantile {i+1}' for i in range(num_cpm_bins)] # e.g., ['Quantile 1', 'Quantile 2', ...]
# Alternative descriptive labels:
# cpm_labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

data['CPM_Category'] = pd.qcut(data['CPM'],
                                q=num_cpm_bins,
                                labels=cpm_labels,
                                duplicates='drop') # Important if many values are the same
data['CPM_Category'] = data['CPM_Category'].cat.add_categories(['No spend'])
condition = (data['Amount_Spent'] == 0) | (data['Amount_Spent'].isna())
data.loc[condition, 'CPM_Category'] = 'No spend'

    
    
# 2. Amount Spent (Buckets of 100s)
# Bins: [0, 100), [100, 200), [200, 300), ...
max_spent = data['Amount_Spent'].max()
# Create bin edges from 0 up to slightly above the max, stepping by 100
amount_bins = np.arange(0, max_spent + 100, 100)
# Create labels like '0-100', '100-200', ...
amount_labels = [f'{int(amount_bins[i])}-{int(amount_bins[i+1])}' for i in range(len(amount_bins)-1)]
data['Amount_Spent_Category'] = pd.cut(data['Amount_Spent'],
                                       bins=amount_bins,
                                       labels=amount_labels,
                                       right=False,
                                       include_lowest=True)
data['Amount_Spent_Category'] = data['Amount_Spent_Category'].cat.add_categories(['No spend'])
condition = (data['Amount_Spent'] == 0) | (data['Amount_Spent'].isna())
data.loc[condition, 'Amount_Spent_Category'] = 'No spend'

print("Amount_Spent binned into 100s.")


# 3. CTR (Buckets of 1%)
# Bins: [0, 1), [1, 2), [2, 3), ...
max_ctr = data['CTR'].max()
ctr_bins = np.arange(0, max_ctr + 1, 1)
ctr_labels = [f'{int(ctr_bins[i])}%-{int(ctr_bins[i+1])}%' for i in range(len(ctr_bins)-1)]
data['CTR_Category'] = pd.cut(data['CTR'],
                              bins=ctr_bins,
                              labels=ctr_labels,
                              right=False,
                              include_lowest=True)
data['CTR_Category'] = data['CTR_Category'].cat.add_categories(['No clicks'])
data['CTR_Category'] = data['CTR_Category'].cat.add_categories(['Invalid CTR'])

condition = (data['CTR'] == 0) | (data['CTR'].isna() | (data['CTR'].isnull()) )
data.loc[condition, 'CTR_Category'] = 'No clicks'

data.loc[data['CTR'] > 1, 'CTR_Category'] = 'Invalid CTR' #There is one record with only 1 impression but 2 clicks. This is possible

print("CTR binned into 1% buckets.")


# 4. Number of Leads (Buckets of 5s)
# Bins: [0, 5), [5, 10), [10, 15), ... (Integers: 0-4, 5-9, 10-14, ...)
max_leads = data['Number_of_Leads'].max()
leads_bins = np.arange(0, max_leads + 5, 5)
leads_labels = [f'{int(leads_bins[i])}-{int(leads_bins[i+1])-1}' for i in range(len(leads_bins)-1)]
data['Leads_Category'] = pd.cut(data['Number_of_Leads'],
                                bins=leads_bins,
                                labels=leads_labels,
                                right=False,
                                include_lowest=True)
print("Number_of_Leads binned into buckets of 5.")


# 5. CPL (Buckets of $10)
# Bins: [0, 10), [10, 20), [20, 30), ...
cpl_nan_label = 'No_Leads'
max_cpl = data['CPL'].max()
cpl_bins = np.arange(0, max_cpl + 10, 10)
cpl_labels = [f'${int(cpl_bins[i])}-${int(cpl_bins[i+1])}' for i in range(len(cpl_bins)-1)]
data['CPL_Category'] = pd.cut(data['CPL'],
                              bins=cpl_bins,
                              labels=cpl_labels,
                              right=False,
                              include_lowest=True)
data['CPL_Category'] = data['CPL_Category'].cat.add_categories(['no leads'])
data.loc[data['Number_of_Leads'] == 0, 'CPL_Category'] = 'no leads'

print("CPL binned into $10 buckets.")

# --- Display Results ---
print("\nDataFrame with new category columns (first 5 rows):")
print(data[['CPM', 'CPM_Category', 'Amount_Spent', 'Amount_Spent_Category', 'CTR', 'CTR_Category', 'Number_of_Leads', 'Leads_Category', 'CPL', 'CPL_Category']].head())


Amount_Spent binned into 100s.
CTR binned into 1% buckets.
Number_of_Leads binned into buckets of 5.
CPL binned into $10 buckets.

DataFrame with new category columns (first 5 rows):
         CPM CPM_Category  Amount_Spent Amount_Spent_Category       CTR  \
0  17.028223   Quantile 1        900.91              900-1000  0.521670   
1  26.748129   Quantile 2        769.20               700-800  0.375559   
2  27.017320   Quantile 2       2125.48             2100-2200  1.069009   
3  11.897764   Quantile 1        626.92               600-700  0.442192   
4  13.273481   Quantile 1        413.23               400-500  1.683156   

  CTR_Category  Number_of_Leads Leads_Category        CPL CPL_Category  
0        0%-1%               56          55-59  16.087679      $10-$20  
1        0%-1%               34          30-34  75.411765      $70-$80  
2  Invalid CTR               32          30-34  66.421250      $60-$70  
3        0%-1%               32          30-34  19.591250      $10-$20  
4

In [32]:
data

,Device_platform,Platform,Use_Custom_Audience,Use_Exclusions,Amount_Spent,CPM,Clicks,Number_of_Leads,CPL,Campaign_Objective,...,Impressions,Reach,Frequency,Currency,CTR,CPM_Category,Amount_Spent_Category,CTR_Category,Leads_Category,CPL_Category
0,mobile_app,instagram,1,0,900.91,17.028223,276,56,16.087679,OUTCOME_LEADS,...,52907,24856.0,2.128540,SGD,0.521670,Quantile 1,900-1000,0%-1%,55-59,$10-$20
1,mobile_app,instagram,0,0,769.20,26.748129,360,34,75.411765,OUTCOME_LEADS,...,95857,36698.0,2.612050,MYR,0.375559,Quantile 2,700-800,0%-1%,30-34,$70-$80
2,mobile_app,facebook,1,1,2125.48,27.017320,841,32,66.421250,OUTCOME_LEADS,...,78671,34312.0,2.292813,SGD,1.069009,Quantile 2,2100-2200,Invalid CTR,30-34,$60-$70
3,mobile_app,instagram,0,0,626.92,11.897764,233,32,19.591250,OUTCOME_LEADS,...,52692,24121.0,2.184487,SGD,0.442192,Quantile 1,600-700,0%-1%,30-34,$10-$20
4,mobile_app,facebook,0,0,413.23,13.273481,524,31,13.330000,OUTCOME_LEADS,...,31132,15056.0,2.067747,SGD,1.683156,Quantile 1,400-500,Invalid CTR,30-34,$10-$20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9743,mobile_app,facebook,0,1,2.97,60.612245,1,0,100000.000000,OUTCOME_LEADS,...,49,48.0,1.020833,SGD,2.040816,Quantile 4,0-100,Invalid CTR,0-4,no leads
9744,mobile_app,instagram,0,1,3.22,89.444444,2,0,100000.000000,OUTCOME_LEADS,...,36,28.0,1.285714,SGD,5.555556,Quantile 4,0-100,Invalid CTR,0-4,no leads
9745,mobile_app,facebook,0,1,7.08,51.304348,7,0,100000.000000,OUTCOME_LEADS,...,138,109.0,1.266055,SGD,5.072464,Quantile 3,0-100,Invalid CTR,0-4,no leads
9746,mobile_app,instagram,0,1,0.49,61.250000,0,0,100000.000000,OUTCOME_LEADS,...,8,8.0,1.000000,SGD,0.000000,Quantile 4,0-100,No clicks,0-4,no leads


# Sensortower Data Preprocessing

In [104]:
df_android = pd.read_csv('./Data/Sensor_Tower_App_Performance_Demographics_2024-10-01_to_2024-12-31_android.csv', sep='\t', encoding='utf-16')
df_ios = pd.read_csv('./Data/Sensor_Tower_App_Performance_Demographics_2024-10-01_to_2024-12-31_ios.csv', sep='\t', encoding='utf-16')
df_android['OS'] = 'android'
df_ios['OS'] = 'ios'
df_platform = pd.concat([df_android, df_ios], ignore_index=True)
df_platform = df_platform[~df_platform['App Name'].str.contains('Lite')]
cols = ['OS','App Name','Female 18-24','Female 25-34','Female 35-44','Female 45-54','Female 55-99','Male 18-24','Male 25-34','Male 35-44','Male 45-54','Male 55-99']
df_platform = df_platform[cols].rename(columns={'App Name': 'Platform'})

age_groups = ['18-24', '25-34', '35-44', '45-54', '55-99']
genders = ['Male', 'Female']
rows = []

# Iterate through each row in the original DataFrame
for _, row in df_platform.iterrows():
    for gender in genders:
        for age_group in age_groups:
            original_column = f'{gender} {age_group}'
            if original_column in df_platform.columns:
                probability = row[original_column]
                if not pd.isna(probability):
                    rows.append({
                        'OS': row['OS'],
                        'Platform': row['Platform'],
                        'Gender': gender,
                        'Age_Group': age_group,
                        'Probability': float(probability.strip('%')) / 100
                    })
                    
df_platform = pd.DataFrame(rows)
df_platform

,OS,Platform,Gender,Age_Group,Probability
0,android,Facebook,Male,18-24,0.1735
1,android,Facebook,Male,25-34,0.2345
2,android,Facebook,Male,35-44,0.1303
3,android,Facebook,Male,45-54,0.0549
4,android,Facebook,Male,55-99,0.0141
5,android,Facebook,Female,18-24,0.1146
6,android,Facebook,Female,25-34,0.1509
7,android,Facebook,Female,35-44,0.0823
8,android,Facebook,Female,45-54,0.0356
9,android,Facebook,Female,55-99,0.0093


# DAG

In [41]:
amt_spent_cat = 'Amount_Spent_Category'
amt_spent = 'Amount_Spent'
cpm = 'CPM_Category'
ctr = 'CTR_Category'
leads = 'Leads_Category' # Corresponds to N
cpl = 'CPL_Category'     # Corresponds to L
device = 'Device_platform' # Corresponds to DP
platform = 'Platform'     # Corresponds to PL
custom_aud = 'Use_Custom_Audience' # Corresponds to CA
exclusions = 'Use_Exclusions'     # Corresponds to EX
objective = 'Campaign_Objective' # Corresponds to CO
hl = 'Headline_Local'
he = 'Headline_Event'
hx = 'Headline_Exclusivity'
hr = 'Headline_Rental'
hrt = 'Headline_Returns' # Assuming maps to HRT
hd = 'Headline_Discounts'
reach = 'Reach'
frequency = 'Frequency'
impressions = 'Impressions'
gender = 'Gender'
age = 'Age'

# List of all nodes expected in the model_data DataFrame
all_nodes = [
    amt_spent, amt_spent_cat, cpm, ctr, leads, cpl, device, platform, custom_aud,
    exclusions, objective, hl, he, hx, hr, hrt, hd, reach, frequency, impressions, gender, age
]


# Elements that can be controlled
headlines = [hl, he, hx, hr, hrt, hd]
settings_context_nodes = [amt_spent_cat, device, platform, custom_aud, exclusions, objective]
all_settings_nodes = list(set(settings_context_nodes + headlines)) # Use set to ensure unique
latent_nodes = [gender, age]

In [42]:
model = DiscreteBayesianNetwork()

print(f"Checking/Adding nodes: {all_nodes}")
# Optional: Check against data columns first
missing_nodes = [node for node in all_nodes if node not in data.columns and node not in latent_nodes]
if missing_nodes:
    print(f"ERROR: Nodes missing from data columns: {missing_nodes}")
    present_nodes = [node for node in all_nodes if node in data.columns]
    print(f"Warning: Adding only nodes present in data: {present_nodes}")
    model.add_nodes_from(present_nodes)
else:
    print("All nodes found. Adding all nodes.")
    model.add_nodes_from(all_nodes) # Add all defined nodes to the model object

# Define Edges (Parent -> Child relationships) based on plausible influence flow
# Using a slightly simplified structure to reduce excessive parent numbers
edges = []
present_model_nodes = list(model.nodes()) # Work only with nodes actually added

# Function to safely add edge if both nodes exist in the model
def add_safe_edge(parent, child):
    if parent in present_model_nodes and child in present_model_nodes and parent != child:
        edges.append((parent, child))

def add_edges(parent_child):
    for parent in parent_child[0]:
        for child in parent_child[1]:
            add_safe_edge(parent, child)

parent_child = [
    [[amt_spent], [amt_spent_cat]], # Amount Spent -> Amount Spent Category
    [[device, platform], latent_nodes], # Device + Platform -> Gender + Age
    [settings_context_nodes + latent_nodes, [cpm]], # Settings + Gender + Age -> CPM (M)
    [latent_nodes + [objective], [reach]], # Gender + Age -> Reach (R)
    [[amt_spent, cpm, objective], [impressions]], # Amount Spent + CPM + Objective -> Impressions
    [[impressions, reach, objective], [frequency]], # Impressions + Reach + Objective -> Frequency
    [settings_context_nodes + headlines + [frequency], [ctr]], # Settings + Headlines + Frequency -> CTR (T)
    [[impressions, ctr], [leads]], # Impressions + CTR -> Leads (N)
    [[amt_spent_cat, leads], [cpl]], # Amount Spent Category + Leads -> CPL (L)
]

add_edges(parent_child)
# Add all defined edges to the model
model.add_edges_from(edges)

Checking/Adding nodes: ['Amount_Spent', 'Amount_Spent_Category', 'CPM_Category', 'CTR_Category', 'Leads_Category', 'CPL_Category', 'Device_platform', 'Platform', 'Use_Custom_Audience', 'Use_Exclusions', 'Campaign_Objective', 'Headline_Local', 'Headline_Event', 'Headline_Exclusivity', 'Headline_Rental', 'Headline_Returns', 'Headline_Discounts', 'Reach', 'Frequency', 'Impressions', 'Gender', 'Age']
All nodes found. Adding all nodes.
